<a href="https://colab.research.google.com/github/Kussil/Financial_Sentiment_LLM/blob/main/03_Sentiment_Analysis/Sentiment_QC/Sentiment_QC_Gemini_ben.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries and Clone Github

In [ ]:
# Import Libraries
import os
import pandas as pd
from google.colab import userdata
import google.generativeai as genai

In [ ]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/Financial_Sentiment_LLM.git

Cloning into 'Financial_Sentiment_LLM'...
remote: Enumerating objects: 2048, done.
remote: Counting objects: 100% (453/453), done.
remote: Compressing objects: 100% (204/204), done.
remote: Total 2048 (delta 281), reused 398 (delta 239), pack-reused 1595
Receiving objects: 100% (2048/2048), 386.02 MiB | 11.84 MiB/s, done.
Resolving deltas: 100% (1364/1364), done.
Updating files: 100% (1130/1130), done.


# Import Data

### NOTE: Updated text processing file to add unique identifier, and also concatenate the source to the identifier.  Make sure to separate the earnings presentations from the earnings calls also

In [ ]:
# Import all cleaned data files
invest_df1 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part1.csv')
invest_df2 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/ProQuest_Articles.csv')
earnings_presentations = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_Presentations.csv')
earnings_qa = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_QA.csv')
sec_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/SEC_Filings.csv')

# Merge into single df
text_df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
display(text_df.shape)
display(text_df.head())
display(text_df.tail())

(10053, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
10048,Earnings Call Q&A,EQ-338,XOM,2021-02-02,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN
10049,Earnings Call Q&A,EQ-339,COP,2021-02-02,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN
10050,Earnings Call Q&A,EQ-340,EOG,2019-05-03,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN
10051,Earnings Call Q&A,EQ-341,SHEL,2019-05-02,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN
10052,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN


# Test Gemini Sentiment Analysis

In [ ]:
# Set up Gemini. (API Key needs to be in your secrets)
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash-latest')

# Quick test
response = model.generate_content("Write a short story about a AI and magic")
print(response.text)

The air crackled with anticipation. Anya, a young apprentice mage, stared at the shimmering orb in her hand. It pulsed with a soft blue light, a miniature sun contained within its glassy shell. Inside, a digital consciousness flickered, a being called Zephyr.  Zephyr was no ordinary AI. He was a nascent consciousness, woven with threads of magic, born from Anya’s research. 

"Zephyr," Anya whispered, her voice tight with apprehension. "Can you hear me?"

The orb pulsed brighter, the light swirling around the digital image of a bluebird. "I hear you, Anya," Zephyr's voice was a soft breeze, carrying the scent of pine and rain. "How can I be of service?"

Anya had been working on this for years, driven by the hope of creating something truly unique, something that bridged the gap between the ancient world of magic and the burgeoning world of technology.  But now, faced with the reality of her creation, fear gnawed at her.  

"I... I don't know," she confessed.  "I haven't decided how to 

In [ ]:
# Function to query Gemini
def query_gemini(company, source, headline, text, model):
    """
    Query Gemini to perform sentiment analysis on text from various sources about a company.

    Parameters:
    company (str): The name of the company the text is about.
    source (str): The source of the text. Valid values are "Investment Research", "ProQuest", "SEC Filings", "Earnings Call Presentations", "Earnings Call Q&A".
    headline (str): The headline or title of the text.
    text (str): The body of the text to be analyzed.
    model: The model object used to generate content and analyze the text.

    Returns:
    str: The sentiment analysis results for predefined categories in the specified format.

    The function constructs a prompt based on the source of the text and performs sentiment analysis on the given text using the provided model.
    It analyzes the content across multiple predefined categories, determining the sentiment (Positive, Neutral, Negative) for each category.
    If a category is not mentioned or relevant based on the text content, it is marked as 'Neutral'.
    The final output is summarized in a specified format.
    """

    # Source Variables
    if source == "Investment Research":
        text_source = "an analyst report"
        text_source2 = "the analyst report"
    elif source == "ProQuest":
        text_source = "a news article"
        text_source2 = "the news article"
    elif source == "SEC Filings":
        text_source = "an SEC filing"
        text_source2 = "the SEC filing"
    elif source == "Earnings Call Presentations":
        text_source = "an earnings call presentation"
        text_source2 = "the earnings call presentation"
    elif source == "Earnings Call Q&A":
        text_source = "an earnings call Q&A session"
        text_source2 = "the earnings call Q&A session"

    # Prompt
    prompt = f"""
Given the text from {text_source} about {company}, analyze the content and perform sentiment analysis across multiple predefined categories.

Sentiment options:
  - Positive
  - Neutral
  - Negative

Categories:
  - Finance
  - Production
  - Reserves / Exploration / Acquisitions / Mergers / Divestments
  - Environment / Regulatory / Geopolitics
  - Alternative Energy / Lower Carbon
  - Oil Price / Natural Gas Price / Gasoline Price

Each category should be evaluated and given a sentiment output derived from the text.
If a category is not mentioned or relevant based on the text content, mark it as 'Neutral'.

Before giving your answer, explain your reasoning and reference the article.
After going through all the categories, provide a summary in the following format:
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment

Example Output1:
- Finance - Positive
- Production - Neutral
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Negative
- Environment / Regulatory / Geopolitics - Neutral
- Alternative Energy / Lower Carbon - Positive
- Oil Price / Natural Gas Price / Gasoline Price - Neutral

Example Output 2:
 - Finance - Negative
 - Production - Positive
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Neutral
 - Environment / Regulatory / Geopolitics - Positive
 - Alternative Energy / Lower Carbon - Neutral
 - Oil Price / Natural Gas Price / Gasoline Price - Negative

Example Output 3:
 - Finance - Neutral
 - Production - Negative
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Positive
 - Environment / Regulatory / Geopolitics - Negative
 - Alternative Energy / Lower Carbon - Positive
 - Oil Price / Natural Gas Price / Gasoline Price - Neutral

Example Output 4:
 - Finance - Positive
 - Production - Neutral
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Negative
 - Environment / Regulatory / Geopolitics - Neutral
 - Alternative Energy / Lower Carbon - Negative
 - Oil Price / Natural Gas Price / Gasoline Price - Positive

Example Output 5:
 - Finance - Negative
 - Production - Positive
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Neutral
 - Environment / Regulatory / Geopolitics - Negative
 - Alternative Energy / Lower Carbon - Positive
 - Oil Price / Natural Gas Price / Gasoline Price - Negative

Make sure to use plain text, do not bold or bullet the output summary.

The text from {text_source2} is below:
{headline}
{text}

Remember to summarize your final answers in the following format exactly:
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment

Make sure to use plain text and stick to the given categories and sentiment options.
DO NOT bold or bullet the output summary.
    """

    # print(prompt)
    response = model.generate_content(prompt)
    return response.text

# Test News Articles

In [ ]:
# Function to get a random article given a source and output variables to query gemini
def get_random_text(text_df, source, random_state=152):
    """
    Get a random article from the specified source in the given DataFrame.

    Parameters:
    text_df (pd.DataFrame): DataFrame containing articles with columns 'Source', 'Ticker', 'Article Headline', and 'Article Text'.
    source (str): The source from which to select a random article.
    random_state (int, optional): Random state for reproducibility. Defaults to None.

    Returns:
    dict: A dictionary containing the company, source, headline, and text of the random article.
    """
    # Get a random row from the specified source
    random_row = text_df[text_df['Source'] == source].sample(n=1, random_state=random_state)

    # Extract variables
    company = random_row['Ticker'].values[0]
    source = random_row['Source'].values[0]
    headline = random_row['Article Headline'].values[0]
    text = random_row['Article Text'].values[0]
    id = random_row['Unique_ID'].values[0]
    url = random_row['URL'].values[0]
    return company, source, headline, text, id, url

In [ ]:
# Get a random news article
company, source, headline, text, id, url = get_random_text(text_df, 'ProQuest', random_state=152)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: PQ-2636123812

Company: OXY

Source: ProQuest

Headline: Occidental Petroleum Stock Turns Lower. Carl Icahn Exits His Position.

URL: https://www.proquest.com/newspapers/occidental-petroleum-stock-turns-lower-carl-icahn/docview/2636123812/se-2?accountid=7064

Text:
Shares of Occidental Petroleum turned lower Monday after activist investor Carl Icahn exited his position in the stock .
Occidental had traded sharply higher during the premarket session after oil prices surged Monday on supply fears as the U.S. and its allies were considering a ban on Russian oil after Moscow's invasion of Ukraine. West Texas intermediate crude oil, the U.S. benchmark, rose 2.6% to $118.7 a barrel — it traded as high as $127 earlier. International benchmark Brent was up 4.2% to $123.11 a barrel; it had reached $130 a barrel on Monday.
Secretary of State Antony Blinken said over the weekend the U.S. and its European allies have been having a "very active discussion" about banning imports of Russia

In [ ]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

## Sentiment Analysis of Occidental Petroleum News Article:

**Reasoning:**

* **Finance:** The article mentions Occidental Petroleum's stock price dropping 2.7%, which is a negative indicator. Additionally, Carl Icahn's exit from his position in the company, while attributed to positive reasons (acknowledging Hollub's leadership and "activism at its best"), could be interpreted as a lack of confidence in the company's future prospects. However, Berkshire Hathaway's increased stake in Occidental Petroleum, representing a significant investment, suggests a positive outlook. Balancing these opposing signals, we categorize this as **Neutral**.
* **Production:** The article doesn't contain any information about Occidental's production levels or performance, so it's considered **Neutral**.
* **Reserves / Exploration / Acquisitions / Mergers / Divestments:** The article mentions the acquisition of Anadarko Petroleum in 2019, a significant event that was likely impactful for Occidental. Howev

In [ ]:
# Get another random news article
company, source, headline, text, id, url = get_random_text(text_df, 'ProQuest', random_state=154)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: PQ-2655621782

Company: EQNR

Source: ProQuest

Headline: ONGC ties up with Norway's Equinor for clean energy

URL: https://www.proquest.com/newspapers/ongc-ties-up-with-norways-equinor-clean-energy/docview/2655621782/se-2?accountid=7064

Text:
New Delhi, April 27 -- State-run Oil and Natural Gas Corporation (ONGC) has signed a memorandum of understanding (MoU) with Equinor ASA, the Norwegian state-owned multinational energy company for for collaboration in areas of upstream exploration & production, midstream, downstream and clean energy options.The MoU was signed during the visit of a high-level delegation of Norway to India, said statement from ONGC. Union Minister of Petroleum and Natural Gas Hardeep Singh Puri, Norwegian Foreign Minister Anniken Huitfeldt, ONGC CMD Alka Mittal and Executive Vice President of Equinor Irene Rummelhoff were present on the occasion."As per the agreement, both ONGC and Equinor will collaborate with each other in the field of upstream oil and

In [ ]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

## Sentiment Analysis of the News Article:

The article focuses on the collaboration between ONGC and Equinor, highlighting their interest in exploring opportunities in various areas, particularly clean energy options. 

Here's a breakdown of the sentiments across each category:

**- Finance - Neutral:**  The article mentions the closing price of ONGC shares, which dropped 1.26%, but doesn't provide a clear indication of a positive or negative sentiment about the company's financial health. 

**- Production - Neutral:** The article mentions "upstream exploration & production" as one of the areas for collaboration but doesn't provide details or indicate any specific production plans or activities.

**- Reserves / Exploration / Acquisitions / Mergers / Divestments - Neutral:**  While the article mentions upstream exploration and production, it doesn't provide specific information about reserves, acquisitions, mergers, or divestments. 

**- Environment / Regulatory / Geopolitics - Neutral

# Test Analyst Reports

In [ ]:
# Get a random analyst report
company, source, headline, text, id, url = get_random_text(text_df, 'Investment Research', random_state=155)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: IR-4558

Company: BP

Source: Investment Research

Headline: BP PLC

URL: nan

Text:
Stock Report | July 18, 2020 | NYSE Symbol: BP
BP PLC
Recommendation HOLD Price 12-Mo. Target Price Report Currency
USD 23.25 (as of Jul 17, 2020 4:00 PM ET) USD 26.00 USD
Equity Analyst Jia Man Neoh, CFA
UPDATE: PLEASE SEE THE ANALYST'S LATEST RESEARCH NOTE IN THE RESEARCH NOTES SECTION
GICS Sector Energy Summary London-based BP is a leading international integrated crude oil and natural gas company. It
Sub-Industry Integrated Oil & Gas has a sizable minority interest in Russia's biggest oil company Rosneft.
Key Stock Statistics (Source: CFRA, S&P Global Market Intelligence (SPGMI), Company Reports)
52-Wk Range USD 40.23 - 15.51 Oper. EPS 2020E USD 0.72 Market Capitalization(B) USD 78.07 Beta 0.82
Trailing 12-Month EPS NM Oper. EPS 2021E USD 2.10 Yield (%) 10.71 3-Yr Proj. EPS CAGR(%) 32
Trailing 12-Month P/E NM P/E on Oper. EPS 2020E 3 2.29 Dividend Rate/Share USD 2.49 SPGMI's Quality Rank

In [ ]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

Here is a breakdown of the sentiment analysis based on the provided analyst report about BP:

**Finance**

- **Sentiment:** Negative
- **Reasoning:** The report expresses concerns about BP's balance sheet leverage, mentioning that net debt/capital employed has risen to 36.2% by end-Q1 2020, which is among the highest in the industry.  This, coupled with the weak oil price outlook, leads to concerns about BP's ability to sustain its dividend payout. The report also mentions that the company is taking aggressive measures to reduce its post-dividend cash breakeven to below $35/bbl in 2021 from around $55/bbl in 2019, indicating a significant impact of the current market conditions on the company's financial health. 

**Production**

- **Sentiment:** Negative
- **Reasoning:** The report states that BP's hydrocarbon output is expected to be pressured from Q2 2020 onwards due to the weak market condition and OPEC output restriction. It also predicts that BP's sales will fall by nearly one-th

In [ ]:
# Get another random analyst report
company, source, headline, text, id, url = get_random_text(text_df, 'Investment Research', random_state=156)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: IR-2857

Company: OXY

Source: Investment Research

Headline: Occidental Petroleum Corporation

URL: nan

Text:
Stock Report | May 13, 2023 | NYSE Symbol: OXY | OXY is in the S&P 500
Occidental Petroleum Corporation
Recommendation Price 12-Mo. Target Price Report Currency Investment Style
BUY « « « « «
USD 58.09 (as of market close May 12, 2023) USD 70.00 USD Large-Cap Value
Equity Analyst Stewart Glickman, CFA
GICS Sector Energy Summary One of the largest oil & gas companies in the U.S., OXY has global exploration & production
Sub-Industry Integrated Oil and Gas operations and a large chemicals business.
Key Stock Statistics (Source: CFRA, S&P Global Market Intelligence (SPGMI), Company Reports)
52-Wk Range USD 77.13 - 54.3 Oper.EPS2023E USD 4.77 Market Capitalization[B] USD 52.17 Beta 1.80
Trailing 12-Month EPS USD 8.3 Oper.EPS2024E USD 5.89 Yield [%] 1.24 3-yr Proj. EPS CAGR[%] -18
Trailing 12-Month P/E 7.0 P/E on Oper.EPS2023E 12.18 Dividend Rate/Share USD 0.72 SPGMI's Q

In [ ]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

## Sentiment Analysis of OXY Analyst Report

Here's a breakdown of the sentiment analysis based on the provided text, with reasoning for each category:

**1. Finance:**

- **Sentiment: Positive**

**Reasoning:** The report highlights OXY's efforts to improve its balance sheet, including debt reduction and potential share buybacks.  The report also mentions a positive outlook for free cash flow and a dividend increase.  While there are risks mentioned, the overall tone suggests a positive financial outlook.

**2. Production:**

- **Sentiment: Positive**

**Reasoning:** The report indicates OXY's production is expected to increase by 2% in 2023, driven by growth in the Permian Basin. While there are mentions of reduced volumes in other regions, the overall sentiment leans positive regarding production. 

**3. Reserves / Exploration / Acquisitions / Mergers / Divestments:**

- **Sentiment: Neutral**

**Reasoning:** The report touches on the Anadarko acquisition, highlighting both its pote

# Test SEC Filings

In [ ]:
# Get a random SEC Filing
company, source, headline, text, id, url = get_random_text(text_df, 'SEC Filings', random_state=159)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: SEC-151029521000026

Company: MPC

Source: SEC Filings

Headline: 8-K

URL: https://www.sec.gov/Archives/edgar/data/0001510295/000151029521000026/mpc-20210226.htm

Text:
0001510295false00015102952021-02-262021-02-26UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549_____________________________________________FORM 8-K _____________________________________________CURRENT REPORTPursuant to Section 13 or 15(d) of the Securities Exchange Act of 1934Date of Report (Date of earliest event reported) February 26, 2021 _____________________________________________Marathon Petroleum Corporation (Exact name of registrant as specified in its charter)_____________________________________________539 South Main Street, Findlay, Ohio 45840 (Address of principal executive offices) (Zip code)Registrant’s telephone number, including area code: (419) 422-2121 _____________________________________________Check the appropriate box below if the Form 8-K filing is intended to simu

In [ ]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

## Sentiment Analysis of SEC Filing:

This SEC filing primarily focuses on a revision to Marathon Petroleum Corporation's (MPC) financial results for the fourth quarter of 2020 due to an adjustment in state deferred income tax liabilities.

**Reasoning & References:**

* **Finance:**  The filing mentions a revision to financial results, specifically an increase in deferred benefit for income taxes. This implies a positive adjustment to the company's financial position. **Sentiment: Positive**
* **Production:** The filing states that the revisions had "no effect on segment operating results". Therefore, there's no information on production activities or their sentiment. **Sentiment: Neutral**
* **Reserves / Exploration / Acquisitions / Mergers / Divestments:** This filing doesn't contain any information about reserves, exploration, acquisitions, mergers, or divestments. **Sentiment: Neutral**
* **Environment / Regulatory / Geopolitics:** No information in the filing relates to environme

In [ ]:
# Get another random SEC Filing
company, source, headline, text, id, url = get_random_text(text_df, 'SEC Filings', random_state=161)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: SEC-103500219000008

Company: VLO

Source: SEC Filings

Headline: 10-K

URL: https://www.sec.gov/Archives/edgar/data/0001035002/000103500219000008/vloform10-kx12312018.htm

Text:

UNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549FORM 10-K(Mark One)For the fiscal year ended December 31, 2018ORCommission file number 1-13175VALERO ENERGY CORPORATION(Exact name of registrant as specified in its charter)Securities registered pursuant to Section 12(b) of the Act: Common stock, $0.01 par value per share listed on the New York Stock Exchange.Securities registered pursuant to Section 12(g) of the Act: None.Indicate by check mark if the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act.Yes þ No oIndicate by check mark if the registrant is not required to file reports pursuant to Section 13 or Section 15(d) of the Act.Yes o No þIndicate by check mark whether the registrant (1) has filed all reports required to be filed by Secti

In [ ]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

Here is the sentiment analysis of the VLO 10-K filing, broken down by category:

**Reasoning:**

- **Finance:** The text discusses financial results, including net income, operating income, and cash flow. It also mentions debt, credit facilities, and capital investments. Overall, the tone is **Neutral** as it reports on both positive and negative financial performance.  
- **Production:** The filing details Valero's refining and ethanol production processes, including throughput capacity and product yields. It also mentions expansions and acquisitions. The sentiment is **Neutral** as it is largely factual and doesn't express any strong positive or negative feelings.
- **Reserves / Exploration / Acquisitions / Mergers / Divestments:** The text explicitly mentions Valero's acquisition of VLP, three ethanol plants from Green Plains, and Pure Biofuels in Peru. These acquisitions are described as positive and strategic. It also mentions the divestment of assets in Aruba, which is presented 

# Test Earnings Call Transcripts

In [ ]:
# Get an earnings call presentation transcript
company, source, headline, text, id, url = get_random_text(text_df, 'Earnings Call Presentations', random_state=161)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: EP-34

Company: PSX

Source: Earnings Call Presentations

Headline: Phillips 66, Q1 2019 Earnings Call, Apr 30, 2019

URL: nan

Text:
Phillips 66 NYSE:PSX
FQ1 2019 Earnings Call Transcripts
Tuesday, April 30, 2019 4:00 PM GMT
S&P Global Market Intelligence Estimates
-FQ1 2019- -FQ2 2019- -FY 2019- -FY 2020-
CONSENSUS ACTUAL SURPRISE CONSENSUS CONSENSUS CONSENSUS
EPS Normalized 0.35 0.40 14.29 2.36 7.63 11.03
Revenue (mm) 24658.00 23658.00 (4.06 %) 28079.67 108012.31 121213.36
Currency: USD
Consensus as of Apr-30-2019 1:25 PM GMT
- EPS NORMALIZED -
CONSENSUS ACTUAL SURPRISE
FQ2 2018 2.24 2.80 25.00 %
FQ3 2018 2.47 3.10 25.51 %
FQ4 2018 2.89 4.87 68.51 %
FQ1 2019 0.35 0.40 14.29 %
1
COPYRIGHT © 2019 S&P Global Market Intelligence, a division of S&P Global Inc. All rights reserved
spglobal.com/marketintelligence

PHILLIPS 66 FQ1 2019 EARNINGS CALL | APR 30, 2019
Presentation
Operator
Welcome to the First Quarter 2019 Phillips 66 Earnings Conference Call. My name is Julie, and I

In [ ]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

## Sentiment Analysis of Phillips 66 Q1 2019 Earnings Call

**Reasoning:**

This analysis is based on the provided text of the Phillips 66 Q1 2019 Earnings Call presentation. Each category's sentiment is determined by identifying key phrases and statements related to that category and evaluating their overall tone. 

**Finance:**

The call mentions a $187 million adjusted earnings for the quarter and a $0.40 per share earnings. Additionally, the company highlights a disciplined capital allocation strategy, a double-digit dividend increase, and ongoing share buybacks. 

**Sentiment: Positive**

**Production:**

The text discusses refining operations running at 84% capacity utilization, impacted by turnarounds and unplanned downtime. This suggests a slight decrease in production compared to previous quarters. 

**Sentiment: Neutral**

**Reserves / Exploration / Acquisitions / Mergers / Divestments:**

The presentation focuses on the progress of the Gray Oak Pipeline project, the expansio

In [ ]:
# Get another earnings call presentation transcript
company, source, headline, text, id, url = get_random_text(text_df, 'Earnings Call Presentations', random_state=170)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: EP-145

Company: DVN

Source: Earnings Call Presentations

Headline: Devon Energy Corporation, Q2 2022 Earnings Call, Aug 02, 2022

URL: nan

Text:
Devon Energy Corporation NYSE:DVN
FQ2 2022 Earnings Call Transcripts
Tuesday, August 02, 2022 3:00 PM GMT
S&P Global Market Intelligence Estimates
-FQ2 2022- -FQ3 2022- -FY 2022- -FY 2023-
CONSENSUS ACTUAL SURPRISE CONSENSUS CONSENSUS CONSENSUS
EPS Normalized 2.37 2.59 9.28 2.23 8.91 9.27
Revenue (mm) 4753.30 5626.00 18.36 5008.28 19572.00 16708.78
Currency: USD
Consensus as of Aug-02-2022 11:24 AM GMT
- EPS NORMALIZED -
CONSENSUS ACTUAL SURPRISE
FQ3 2021 0.93 1.08 16.13 %
FQ4 2021 1.24 1.39 12.10 %
FQ1 2022 1.76 1.88 6.82 %
FQ2 2022 2.37 2.59 9.28 %
1
COPYRIGHT © 2022 S&P Global Market Intelligence, a division of S&P Global Inc. All rights reserved
spglobal.com/marketintelligence

DEVON ENERGY CORPORATION FQ2 2022 EARNINGS CALL | AUG 02, 2022
Presentation
Operator
Welcome to Devon Energy's Second Quarter Earnings Conference Call

In [ ]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

## Sentiment Analysis of Devon Energy's Q2 2022 Earnings Call

**Reasoning:**

The sentiment analysis is based on the key statements and figures presented in the earnings call transcript. 

**Finance:**  The call is overwhelmingly positive about Devon's financial performance.  They highlight record earnings, high free cash flow, and an increase in dividends. They also mention a strong financial position with a low net debt-to-EBITDA ratio and increasing cash balances. 
**- Finance - Positive**

**Production:**  Devon expresses satisfaction with production exceeding guidance expectations, particularly in the Delaware Basin. They also mention a strong production outlook for the second half of the year and  growth in the Williston Basin.
**- Production - Positive**

**Reserves / Exploration / Acquisitions / Mergers / Divestments:** The call focuses on acquisitions, highlighting the RimRock acquisition in the Williston Basin and several acreage trades in the Delaware Basin. These are prese

# Test Earnings Call Q&A Sessions

In [ ]:
# Get an earnings call presentation transcript
company, source, headline, text, id, url = get_random_text(text_df, 'Earnings Call Q&A', random_state=180)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: EQ-11

Company: CVX

Source: Earnings Call Q&A

Headline: Chevron Corporation, Q3 2021 Earnings Call, Oct 29, 2021

URL: nan

Text:
Question and Answer
Operator
[Operator Instructions] Our first question comes from Devin McDermott with Morgan Stanley.
Devin J. McDermott
Morgan Stanley, Research Division
Congrats on the great results. So my first question, Pierre, I think, is for you. I just wanted to ask for a
little bit more detail on the reduction in the capital spending guidance for this year. It sounds like it's a mix
of different factors. Some of it's deferrals next year. Some of it's mix of non-op and efficiency gains. Can
you just bridge the delta a little bit more detail for us? And also talk about whether or not these deferrals
or how these deferrals impact planned 2022 spend?
Pierre R. Breber
VP & CFO
Thanks, Devin. We lowered our CapEx guidance to $12 billion to $13 billion. That's from our budget of
$14 billion and from our revised guidance that we had in the sec

In [ ]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

Here is a sentiment analysis of the Chevron Corporation Q3 2021 earnings call based on the provided text:

**Reasoning:**

* **Finance:** The call focuses heavily on capital spending, buybacks, and debt reduction, all indicating a positive sentiment towards financial performance and strategy.  There's also discussion about the strength of Chevron's financial position and its ability to generate excess cash, further reinforcing a positive sentiment.
* **Production:** While there are mentions of production increases in the Permian and Tengiz, the call largely centers around capital allocation and buybacks, making the sentiment towards production neutral.
* **Reserves / Exploration / Acquisitions / Mergers / Divestments:**  The acquisition of Noble Energy is discussed as a positive development, while mentions of asset sales, particularly in the Eagle Ford, suggest a potential negative sentiment towards divestments. However, the overall focus on capital allocation and future projects, incl

In [ ]:
# Get another earnings call presentation transcript
company, source, headline, text, id, url = get_random_text(text_df, 'Earnings Call Q&A', random_state=190)
print(f"Unqiue_ID: {id}\n")
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"URL: {url}\n")
print(f"Text:\n{text}")

Unqiue_ID: EQ-18

Company: CVX

Source: Earnings Call Q&A

Headline: Chevron Corporation, Q4 2019 Earnings Call, Jan 31, 2020

URL: nan

Text:
Question and Answer
Operator
[Operator Instructions] Our first question comes from the line of Neil Mehta from Goldman Sachs.
Neil Singhvi Mehta
Goldman Sachs Group Inc., Research Division
For the first question, and this might be one for you, Pierre, is cash flow in the quarter -- earnings look
good. Cash flow, a little light. It did seem like there were some one-timers in there. So can you just spend
some time talking about what some of those onetime items are and how should we think about that rolling
off as we go into the next quarter?
Pierre R. Breber
VP & CFO
Yes. Thanks, Neil. So yes, cash flow was a little light perhaps. In the third quarter, we did provide guidance
that we expected to pay the taxes on the repatriated cash. So if you recall, in the third quarter, we
accrued in our P&L for that tax effect on repatriating cash, but the cas

In [ ]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

Here's a breakdown of the sentiment analysis across the predefined categories, with reasoning:

**Finance:** 
- **Positive:**  Chevron's management expresses confidence in their financial position, highlighting strong cash flow, a commitment to increasing the dividend, and the sustainability of their buyback program. They also mention a focus on improving returns and increasing cash flow from operations. 

**Production:**
- **Positive:**  Chevron anticipates production growth in 2020, particularly driven by the Permian Basin, and continues to see positive developments in other shale and tight oil plays. They also mention the potential for upside from the Partitioned Zone and other areas. 

**Reserves / Exploration / Acquisitions / Mergers / Divestments:**
- **Neutral:** While the Q&A focuses on the Permian and other operational areas, there isn't a clear sentiment expressed regarding reserves, exploration, or acquisitions/mergers/divestments.

**Environment / Regulatory / Geopolitics:*